(filter:filters)=
# Filtering

Filter parameters return (in comparision to search paramaters) more precise matching on object records of interest, by only returning those object records that exactly match the term(s) you have specified on a single field (or across a group of related fields), instead of matching in any field (as general search using "q" parameter does). In addition, these fields are vocabulary controlled (that is, instead of typing in the word to search for, we use an identifier instead such as "A101", or "THES12345" and so on) which avoids issues with variations in spelling (as search on a single field such as "q_materials_techniques" will need to deal with). Whilst this way of finding records is always preferred if possible, it does mean you need to know the right identifier to use. See XXX

## Quickstart

Let's go through a simple example before we go through filtering in more detail

In [9]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=AAT45514')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that have been catalogued as using the material 'Jet' (AAT45514)")

There are 50 objects that have been catalogued as using the material 'Jet' (AAT45514)


As before, we are using the object search endpoint (further on we will cover combining search and filtering parameters in one query). But we are no longer passing a 'q' parameter for a general record search, we now pass a parameter that starts with 'id_', standing for identifier. This means we only want object records returned that, in the field(s) applicable to a material ('materials') that are vocabulably controlled, have the same identifier, that is, object records that say this material is part of the object.

### Filtering vs Constrained Search vs Full Search

Let's compare the results from filtering with a general text search and a constrained text search. We know from the filter example above the number of results by filtering, lets see what we get with a constrained search for the word 'Jet' as a material or technique

In [ ]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?q_material_technique=Jet')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that mention the term 'Jet' within materials and technique")

Finally, let's search for the term 'Jet' appearing anywhere in the record using a general text search:

In [10]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?q=Jet')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that mention the term 'Jet' anywhere in the record")

There are 506 objects that mention the term 'Jet' anywhere in the record


So, as we can see from the numbers, as probaly expected there are many more records using the term 'Jet' somewhere within the record than are found by searching specifically on the material or technique 'Jet' (or even more precisely by filtering using identifier 'AAT45514' ).  This could be for a few different reasons:
    
    * Some of the matches on the general search for the term 'Jet' are using it with other meanings, such as Jet Engine, Design for Jet Jewellery. These might be irrelvant to your search for objects made of Jet (although possibly the designs might also be of interest, this is where XXX)
    * Some objects which are made of Jet have object records that have not been extensively catalogued, but they do mention in another (free-text) fields the object is made of Jet. These objects would be excluded from your search only on material and technique field, or filtering on the identifer for Jet (AAT45514)
    * Some objects are made of a sub-type of Jet, such as XXX. These would not be found by filtering on Jet (AAT45514), as the identifier used would not match. For this to work (which is an future enhancement to the API), we would need some form of hierarchical filtering to enable matching on a type (Jet) and all its sub-types (.. Jet, .. Jet, etc) and their sub-types, and so on.
    
So, in summary, while filtering on the identifier (the first query we ran) will provide the most precise results, it is likely to not return *all* relevant results.  Keep this in mind when you use filtering, that although the number returned gives the appearance of an exact count (e.g. the V&A holds 50 objects made of Jet), you should also compare this result with other searches (using 'q' and 'q_material_techniques'), to see if there are other objects not yet catalogued as extensively that would also match. And also, remember cataloguing, is a never ending process, so this number is likely to change over time.

## Common Filter Operations 

Some issues and techniques are applicable across all filters

### Finding the identifer you need to use

Certainly the most obvious question, how to find the right identifier to filter on. Beyond having memorised the Getty vocabularies (and the local V&A identifiers) there is no exact science to this. We have listed some common identifiers here to get you started (XXX), but after that you will need to explore the site (looking at URLS, XX) or execute exploratorer searches and find the idenfiers in the objects returned.

Be particular aware that some vocabularies are hierarchical, so can contain more precise deinfitions of some terms, ensure you are using the right .. XXX (and see the note about hierarchical filtering above)

### Matching on multiple filters of the same facet

You might want to filter on object records that contain for example, both silver and lace. This is done fairly simply by repeating the filter parameter with different values:


In [14]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=AAT11029&id_material=AAT132861')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that contain silver ('AAT11029') and/or lace ('AAT132861')")

There are 9791 objects that contain silver ('AAT11029') and/or lace ('AAT132861')


(filter:narrower)=
### Narrower Matching (Multiple filters of the same facet)

As you might notice from the above result, the object records returned will be those that say the object is made with silver and lace, but it will also include those that only say the object was made with lace (with no mention of silver) and likewise for those object made with silver (with no mention of lace)

You might instead want to filter on object records that contain *only* both silver and lace. This is done as above, but you also need to set the search_mode
parameter to "narrow" (otherwise known as boolean AND) instead of the default mode of "broad" (otherwise known as boolean OR).

In [15]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=AAT11029&id_material=AAT132861&search_mode=narrow')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that contain silver ('AAT11029') and lace ('AAT132861')")

There are 5 objects that contain silver ('AAT11029') and lace ('AAT132861')


Please note that this search mode setting then applies to all facets specified, if you are querying more than one. You cannot set only some facets search mode to broadening.

### Negation

You can also use filters to *exclude* any object records that would match the term(s) you have specified, by putting '-' before the identifier. So for example, if you want all object records that are not made of plastic:

In [5]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=-AAT14570&id_material=AAT10797&search_mode=narrow')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that do not contain plastic ('AAT14570') and do contain glass ('AAT10797')")

There are 5571 objects that do not contain plastic ('AAT14570') and do contain glass ('AAT10797')


In [6]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=AAT14570&id_material=AAT10797&search_mode=narrow')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that do contain plastic ('AAT14570') and do contain glass ('AAT10797')")

There are 68 objects that do contain plastic ('AAT14570') and do contain glass ('AAT10797')


```{note}
Please note, use of negation automatically enables a narrow search mode (see {ref}`filter:narrower`), as we do not think there is use case for negation and broad searching. If you do have one, please let us know``` 

## Filters 

(filter:material)=
### Filter by material

As covered already in the first example above, this lets you find object records cataloging some material as part of the object.

Identifiers in use come from the Getty Art & Architecture Thesarus (identifier starting with AAT), and local identifiers from the V&A (identifiers starting with THES)

In [18]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=AAT10797')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that have been catalogued as using the material 'Glass' (AAT10797)")

There are 5639 objects that have been catalogued as using the material 'Glass' (AAT10797)


(filter:technique)=
### Filter by technique

Very similiar to materials, this filter lets you find catalogued object records making use of a particular production technique.

In [7]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_technique=AAT53241')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that use production technique 'etching' (AAT300053241)")

There are 32943 objects that use production technique etching (AAT300053241)


(filter:style)=
### Filter by style

This lets you filter object records by the artistic style

In [8]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_style=AAT21515')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that use style 'vorticism' (AAT300021515) as an artistic classification")

There are 20 objects that use style vorticism (AAT300021515) as an artistic classification


(filter:organisation)=
### Filter by organisation

This lets you filter object records by the organisation involved in the object production

In [12]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_organisation=AUTH339709')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that were production by the Underground London Electric Railway Company organisation (AUTH339709)")

There are 38 objects that were production by the Underground London Electric Railway Company organisation (AUTH339709)


(filter:person)=
### Filter by person

This letd you filter object records by the person involved in the objects production

In [13]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_person=A6630')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print("There are {record_count} objects that are connected in some way to J.B. Yeats (A6630)")

There are 12 objects that are connected to J.B. Yeats (AUTH339709)


(filter:depiction)=
### Filter by depiction

This lets you filter object records by the person, place, concept depicted in the object

In [20]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_depicts=A6630')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} object(s) that depict J.B. Yeats (A6630)")

There are 1 object(s) that depict J.B. Yeats (A6630)


(filter:maker)=
### Filter by maker

This lets you filter object records by the maker of the object (this can be a person, a group of people, or an organisation) 

In [14]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_maker=A6630')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that are made by J.B. Yeats (A6630)")

There are 11 objects that are connected to J.B. Yeats (AUTH339709)


(filter:place)=
### Filter by place

This lets you filter object records by a place that is where the object was made, depicts or is otherwise associated with.

In [19]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_place=x37151')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that are from, depict or are associated with the place 'Redcar and Cleveland'")

There are 7 objects that are associated with the place 'Redcar and Cleveland'


category(filter:category)=
### Filter by category

This lets you filter object records by a categorisation applied to the object.

In [22]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_category=THES49002')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that are categorised as 'Gardens & Gardening'")

There are 2252 objects that are categorised as 'Gardens & Gardening'


(filter:collection)=
### Filter by collection

This lets you filter object records by the collecting department in the museum that manages the object. XXX List departments ?

In [23]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_collection=THES260586')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that are part of the 'Digital, Architecture and Design' department")

There are 796 objects that are part of the 'Digital, Architecture snd Design' department


(filter:association)=
### Filter by association

This lets you filter object records by a person, place, people, organisation associated with the object.

In [27]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_associated=A8575')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} object(s) with which William Kent (A8575) is associated (but not neccessarily as the maker of the object and/or depicted in the object)")

There are 1 object(s) with which William Kent (A8575) is associated (but not neccessarily as the maker of the object and/or depicted in the object)


### Filter by accession year

In [2]:
import requests
req = requests.get('http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_accession_year=1919')
object_data = req.json()
object_info = object_data["info"]
object_records = object_data["records"]
record_count = object_info["record_count"]
print(f"There are {record_count} objects that were accessioned into the collection in 1919")

There are 10101 object(s) that were accessioned into the collection in 1919


## Advanced Topics

### Combining filters with search

It might seem strange after talking about the precision of filtering to bring general search back into a query as well, but the combination of filtering and search is the most common
XXX (collections web interface)

### Boolean support within filtering

You might be thinking another way to exactly specify multiple filters is to provide support for the boolean operators available to search parameters within a filter param. For example using id_person="A6630|A6631" to indicate you want object records mentioning either A6630 or A6631 or both of them. You might also think you could further extend this to even more complex filtering by using boolean operators in multiple parameters (id_person, id_material, etc); to specify the way you want filters to be combined (e.g. applying boolean AND between facets ('tightening', boolean OR within facets ('broadening'), except for materials facet where boolean negation applies, except for material type silver which must be present on all matching object records. You would rightly think this would then provide you with a more granualar way to control search mode, per facet instead of forcing them all to be "broad" or "narrow" as the "search_mode" parameter does.

You might think all this, but at present it is not supported.
